In [1]:
%%bash
pip install pyyaml -q

In [2]:
import os
import cv2
import yaml
import random as r
import numpy as np
from yaml.loader import SafeLoader

BASE_PATH: str = "../input/playing-cards-object-detection-dataset"

In [3]:
for folder_name in ["train", "valid", "test"]:

    img_filenames = [filename[:-4] for filename in sorted(os.listdir(BASE_PATH + f"/{folder_name}/images"))]
    txt_filenames = [filename[:-4] for filename in sorted(os.listdir(BASE_PATH + f"/{folder_name}/labels"))]

    assert img_filenames == txt_filenames, f"{folder_name} Failure"

In [4]:
with open("../input/playing-cards-object-detection-dataset/data.yaml") as f:
    yaml_data = yaml.load(f, Loader=SafeLoader)
class_names = yaml_data["names"]

In [5]:
# with open("../input/playing-cards-object-detection-dataset/train/labels/000090528_jpg.rf.d50e89610e5c97c61632c290692f3e75.txt") as f:

path = "../input/playing-cards-object-detection-dataset/train/labels"
name = os.listdir(path)[r.randint(0, len(os.listdir(path))-1)]

with open(os.path.join(path, name)) as f:
    data = f.read()
    
data = [data_item.split(" ") for data_item in data.split("\n")]
data

[['47',
  '0.15384615384615385',
  '0.43509615384615385',
  '0.09975961538461539',
  '0.06370192307692307'],
 ['11',
  '0.2091346153846154',
  '0.33774038461538464',
  '0.09615384615384616',
  '0.07932692307692307'],
 ['14',
  '0.2800480769230769',
  '0.2692307692307692',
  '0.07211538461538461',
  '0.078125'],
 ['14',
  '0.6983173076923077',
  '0.32211538461538464',
  '0.07692307692307693',
  '0.078125']]

### **Dataset and DataLoader Analysis**

In [6]:
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL

from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence

In [7]:
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)
    return image

In [8]:
class DS(Dataset):
    def __init__(
        self, 
        base_path: str="../input/playing-cards-object-detection-dataset/valid/images",
        filenames: np.ndarray=None, 
        labels: list=None, 
        bboxes: list=None,
        size: int=224,
        transform: transforms=transforms.Compose([transforms.ToTensor(),])
    ):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.bboxes = bboxes
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]

    def __getitem__(self, idx):
        image = get_image(os.path.join(self.base_path, self.filenames[idx] + ".jpg"), self.size)
        if self.transform:
            image = self.transform(image)
        label = torch.LongTensor(self.labels[idx])
        bbox  = torch.FloatTensor(self.bboxes[idx])
        
        return image, label, bbox

In [9]:
path = "../input/playing-cards-object-detection-dataset/valid/labels"
names = np.array([name[:-4] for name in sorted(os.listdir(path))])

final_classes: list = []
final_bboxes : list = []
        
for name in names:
    classes: list = []
    bboxes: list  = []
    with open(os.path.join(path, name + ".txt")) as f:
        data = f.read()
    
    data = [data_item.split(" ") for data_item in data.split("\n")]
    
    for item in data:
        classes.append(int(item[0]))
        bboxes.append([
            float(item[1]), 
            float(item[2]), 
            float(item[3]), 
            float(item[4]),
        ])
    
    final_classes.append(classes)
    final_bboxes.append(bboxes) 

In [10]:
def my_collate_fn(batch):
    label_list: list = []
    bbox_list: list  = []
    images = torch.FloatTensor(len(batch), 3, 224, 224)
    
    i = 0
    for image_, label_, box_ in batch:
        images[i] = image_
        label_list.append(label_)
        bbox_list.append(box_)       
    
    label_list = pad_sequence(label_list, batch_first=True, padding_value=99)
    bbox_list = pad_sequence(bbox_list, batch_first=True, padding_value=99)

    return images, label_list, bbox_list

In [11]:
va_data_setup = DS(filenames=names, labels=final_classes, bboxes=final_bboxes)
va_data = DL(va_data_setup, batch_size=16, shuffle=False, collate_fn=my_collate_fn)

In [12]:
imgs, lbls, bbxs = next(iter(va_data))

print(f"imgs.shape    : {imgs.shape}")
print(f"imgs[0].shape : {imgs[0].shape}")
print(f"lbls.shape    : {lbls.shape}")
print(f"bbxs.shape    : {bbxs.shape}")

imgs.shape    : torch.Size([16, 3, 224, 224])
imgs[0].shape : torch.Size([3, 224, 224])
lbls.shape    : torch.Size([16, 4])
bbxs.shape    : torch.Size([16, 4, 4])
